# Naive representation pipeline (qiskit)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

Data embedding:

* circuit family #1 (controlled-X) from https://arxiv.org/abs/1803.01958

In [5]:
import numpy as np
import sys
import struct

import qiskit
from qiskit import *

## Generating an artificial image of the sky

In [9]:
#image of the sky filled with double precision complex floats (32 bits for real/imaginary parts)
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[9.48944855e+01+3.5098068e+01j 9.48944855e+01+3.5098068e+01j
  4.96364152e-03+5.6190095e-03j 9.48944855e+01+3.5098068e+01j]
 [9.48944855e+01+3.5098068e+01j 1.88531098e-03+9.6744928e-04j
  1.27701159e-03+1.7982314e-03j 1.83291442e-03+5.7927743e-03j]
 [1.46740858e-04+7.1737766e-03j 5.56186493e-03+4.3277629e-03j
  7.38066109e-03+8.8657467e-03j 7.82662537e-05+6.7799361e-03j]
 [9.48944855e+01+3.5098068e+01j 1.01912316e-04+9.1597969e-03j
  7.03828176e-03+8.9817103e-03j 9.89230350e-03+6.3023106e-03j]]


## Applying 2D Fourier transform (visibilities)

In [10]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 474.51257 +175.5561j    284.6585  +105.280365j   94.89594  +35.097176j
   284.66736 +105.27186j ]
 [ 284.65936 +105.28471j    94.896545 +35.101547j  -94.88989  -35.097897j
    94.91133  +35.075214j]
 [  94.89059  +35.09783j   -94.89885  -35.10982j  -284.6712  -105.27221j
   -94.89788  -35.079376j]
 [ 284.69113 +105.26064j    94.90187  +35.097702j  -94.89293  -35.096867j
    94.87727  +35.1021j  ]]


### Sanity check

In [11]:
img = np.fft.ifft2(visibilities).astype('complex64')

print(img)

test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[9.4894485e+01+3.5098068e+01j 9.4894485e+01+3.5098068e+01j
  4.9633980e-03+5.6192875e-03j 9.4894485e+01+3.5098068e+01j]
 [9.4894485e+01+3.5098068e+01j 1.8856525e-03+9.6702576e-04j
  1.2784004e-03+1.7983913e-03j 1.8317699e-03+5.7935715e-03j]
 [1.4638901e-04+7.1732998e-03j 5.5611134e-03+4.3263435e-03j
  7.3819160e-03+8.8655949e-03j 7.7009201e-05+6.7801476e-03j]
 [9.4894485e+01+3.5098068e+01j 1.0037422e-04+9.1590881e-03j
  7.0385933e-03+8.9814663e-03j 9.8922253e-03+6.3028336e-03j]]

Reals MSE: 6.079587e-13
Imaginaries MSE: 2.539201e-13


## Classical data encoding/decoding

In [12]:
number_of_entries = visibilities.shape[0] * visibilities.shape[1]
number_of_bits_real = 32 #since single precision complex floats
number_of_bits_imag = 32
number_of_bits = number_of_bits_real + number_of_bits_imag
off_set = 0

qc = QuantumCircuit(number_of_entries*number_of_bits) #n bits encoded into n qbits (circuit family #1)
for i in range(0, visibilities.shape[0]):
    for j in range(0, visibilities.shape[1]):
            binary_real = float_to_bin_real(visibilities[i, j].real)
            binary_imag = float_to_bin_imag(visibilities[i, j].imag)
            binary = binary_real+binary_imag
            off_set = encoding2(qc, binary, off_set)

#measurement
qc.measure_all()
backend = Aer.get_backend('aer_simulator')
job = backend.run(qc, shots=1, memory=True)
output = job.result().get_memory()[0]
out = reverse(output)

#readout
chunks_real = []
chunks_imag = []
for i in range(0, number_of_entries):
    chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) #real parts represented every 64 bits starting from the first one
    chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) #imaginary parts represented every 64 bits starting after the first occurence of a real part 

readout = []
for i in range(0, len(chunks_real)):
    readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

#reshaping the readout vector into a nxn matrix
readout = np.array(readout).reshape(n , n).astype('complex64')
print(readout)

[[ 474.51257 +175.5561j    284.6585  +105.280365j   94.89594  +35.097176j
   284.66736 +105.27186j ]
 [ 284.65936 +105.28471j    94.896545 +35.101547j  -94.88989  -35.097897j
    94.91133  +35.075214j]
 [  94.89059  +35.09783j   -94.89885  -35.10982j  -284.6712  -105.27221j
   -94.89788  -35.079376j]
 [ 284.69113 +105.26064j    94.90187  +35.097702j  -94.89293  -35.096867j
    94.87727  +35.1021j  ]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [13]:
img = np.fft.ifft2(readout).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[9.4894485e+01+3.5098068e+01j 9.4894485e+01+3.5098068e+01j
  4.9633980e-03+5.6192875e-03j 9.4894485e+01+3.5098068e+01j]
 [9.4894485e+01+3.5098068e+01j 1.8856525e-03+9.6702576e-04j
  1.2784004e-03+1.7983913e-03j 1.8317699e-03+5.7935715e-03j]
 [1.4638901e-04+7.1732998e-03j 5.5611134e-03+4.3263435e-03j
  7.3819160e-03+8.8655949e-03j 7.7009201e-05+6.7801476e-03j]
 [9.4894485e+01+3.5098068e+01j 1.0037422e-04+9.1590881e-03j
  7.0385933e-03+8.9814663e-03j 9.8922253e-03+6.3028336e-03j]]

Reals MSE: 6.079587e-13
Imaginaries MSE: 2.539201e-13


## Utils

### Quantum

In [4]:
def encoding2(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i)

        if binary[i]=='1':
            qc.x(off_set+i)
    
    off_set += len(binary)

    return off_set

### Classical

In [3]:
#float to binary / binary to float
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

#reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string

#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask